In [1]:
import numpy as np # linear algebra
import pandas as pd

In [2]:
user_hist_df = pd.read_csv('../data/raw/movies_dataset_10 months.csv')
user_hist_df['ts'] = pd.to_datetime(user_hist_df['ts'])
user_hist_df['month'] = user_hist_df['ts'].dt.month

test_1 = user_hist_df[user_hist_df['month'] == 6]
test_2 = user_hist_df[user_hist_df['month'] == 7]
train = user_hist_df[~user_hist_df['month'].isin([6, 7])]

In [3]:
correct_1 = test_1.drop_duplicates(['user_id', 'movie_id']).groupby('user_id')['movie_id'].apply(list).to_dict()
correct_2 = test_2.drop_duplicates(['user_id', 'movie_id']).groupby('user_id')['movie_id'].apply(list).to_dict()


In [4]:
train['event'] = 1

/Users/danil/Documents/github/sweet_RS/env/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [5]:
train = train.drop_duplicates(['user_id', 'movie_id', 'event'])

In [6]:
train.movie_id = train.movie_id.astype('category')
train['categ_id'] = train.movie_id.cat.codes + 1

In [7]:
# Let`s define inverse transform dictionary
cat_to_element_uid = dict(zip(
    range(1, len(train.movie_id.cat.categories) + 1),
    train.movie_id.cat.categories
))

# Assigning most popular film index to inverse transform of zero padding value
cat_to_element_uid[0] = 18943

In [10]:
%%time
import tqdm
tqdm.tqdm.pandas()
sequences = train.groupby('user_id')['categ_id'].progress_apply(list)

100%|██████████| 2055/2055 [00:00<00:00, 11979.86it/s]

CPU times: user 161 ms, sys: 16.6 ms, total: 178 ms
Wall time: 193 ms


In [11]:
sequences.head()

user_id
446882394249262      [1003, 782, 1148, 2692, 944, 3451, 1, 960, 991...
11853380549289372    [3581, 148, 1575, 392, 1696, 1641, 2979, 2288,...
29584184913820301    [3695, 3687, 3150, 2788, 541, 2318, 1073, 2852...
34799051736481083    [2622, 319, 1753, 1463, 2692, 977, 1278, 379, ...
69093689723640662    [3025, 2837, 1801, 2561, 106, 2462, 2727, 860,...
Name: categ_id, dtype: object

In [12]:
# Some statistics
print('Median length: {}\nMean length: {}\nMax length: {}'.format(
    sequences.apply(len).median(), sequences.apply(len).mean(), sequences.apply(len).max()))

Median length: 82.0
Mean length: 99.8374695863747
Max length: 1009


In [13]:
sequences2use = sequences[sequences.apply(len) >= 50]

In [14]:
maxlen = 90 # Length of sequences in X
X = []
y = []

def slice_sequence(seq, num_slices):
    for i in range(1, num_slices):
        X.append(seq[-(i+maxlen): -i])
        y.append(seq[-i])
        
for seq in tqdm.tqdm(sequences2use):
    if len(seq) <= 70:
        slice_sequence(seq, 10)
    elif len(seq) <= 90:
        slice_sequence(seq, 15)
    elif len(seq) <= 100:
        slice_sequence(seq, 20)
    else:
        slice_sequence(seq, 25)

100%|██████████| 1542/1542 [00:00<00:00, 18951.89it/s]


In [15]:
len(X), len(y)

(28443, 28443)

In [16]:
lens = [len(x) for x in X]
max(lens), min(lens), np.mean(lens), np.median(lens)

(90, 41, 83.12702598178814, 90.0)

In [21]:
pip install tensorflow

     |████████████████████████████████| 165.1MB 64kB/s  eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/1a/8b/4d01ae9a9d50a0bcc7b0b9aae41785d8d9de6fa9bba04dc20b1582181d2d/h5py-2.10.0-cp37-cp37m-macosx_10_6_intel.whl
     |████████████████████████████████| 10.6MB 440kB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/8a/48/a76be51647d0eb9f10e2a4511bf3ffb8cc1e6b14e9e4fab46173aa79f981/termcolor-1.1.0.tar.gz
     |████████████████████████████████| 51kB 1.0MB/s eta 0:00:011
  Using cached https://files.pythonhosted.org/packages/3e/00/0266fefaafb839760d5b25b884375b2ab0f842ebe138ee6c1ef807af44bb/numpy-1.18.5-cp37-cp37m-macosx_10_9_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/82/f7/e43cefbe88c5fd371f4cf0cf5eb3feccd07515af9fd6cf7dbf1d1793a797/wrapt-1.12.1.tar.gz
     |████████████████████████████████| 71kB 784kB/s eta 0:00:01
     |████████████████████████████████| 133kB 517kB/s eta 0:00:01
     |████████████████████████████████| 3.4MB

  Using cached https://files.pythonhosted.org/packages/05/57/ce2e7a8fa7c0afb54a0581b14a65b56e62b5759dbc98e80627142b8a3704/oauthlib-3.1.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/62/1e/a94a8d635fa3ce4cfc7f506003548d0a2447ae76fd5ca53932970fe3053f/pyasn1-0.4.8-py2.py3-none-any.whl
  Found existing installation: numpy 1.19.4
    Uninstalling numpy-1.19.4:
      Successfully uninstalled numpy-1.19.4
  Found existing installation: h5py 3.1.0
    Uninstalling h5py-3.1.0:
      Successfully uninstalled h5py-3.1.0
  Running setup.py install for termcolor ... done
  Running setup.py install for wrapt ... done
You should consider upgrading via the 'pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [22]:
from keras.preprocessing.sequence import pad_sequences

# We should pad our sequences with 0 values, so they all will have the same length
X = pad_sequences(X, maxlen=maxlen)
y = np.array(y)
X.shape, y.shape

((28443, 90), (28443,))

In [25]:
from keras.layers import Input, Embedding, SpatialDropout1D, LSTM, Dropout, Dense
from keras.models import Model

# Let's set random seed
import tensorflow as tf
#tf.set_random_seed(42)
np.random.seed(42)

In [26]:
train.categ_id.unique().size + 1

4733

In [30]:
max_features = train.categ_id.unique().size + 1
embed_size = 64

def lstm128():
    inp = Input(shape=(maxlen,))
    x = Embedding(max_features, embed_size)(inp)
    x = SpatialDropout1D(0.05)(x)
    x = LSTM(128, return_sequences=False)(x)
    x = Dropout(0.02)(x)
    outp = Dense(max_features, activation="softmax")(x)
    model = Model(inputs=inp, outputs=outp)
    model.compile(loss='sparse_categorical_crossentropy', optimizer='rmsprop',
                  metrics=['sparse_categorical_accuracy'])
    return model

In [31]:
model = lstm128()
model.fit(X, y, batch_size=2048*4, epochs=25, verbose=True, validation_split=0.01, shuffle=True)

Epoch 1/25
4/4 [==============================] - 51s 13s/step - loss: 8.4530 - sparse_categorical_accuracy: 0.0014 - val_loss: 8.0561 - val_sparse_categorical_accuracy: 0.0035
Epoch 2/25
4/4 [==============================] - 53s 13s/step - loss: 7.7638 - sparse_categorical_accuracy: 0.0026 - val_loss: 7.4410 - val_sparse_categorical_accuracy: 0.0105
Epoch 3/25
4/4 [==============================] - 35s 9s/step - loss: 7.3531 - sparse_categorical_accuracy: 0.0063 - val_loss: 7.3411 - val_sparse_categorical_accuracy: 0.0105
Epoch 4/25
4/4 [==============================] - 33s 8s/step - loss: 7.2704 - sparse_categorical_accuracy: 0.0072 - val_loss: 7.3157 - val_sparse_categorical_accuracy: 0.0070
Epoch 5/25
4/4 [==============================] - 33s 8s/step - loss: 7.2460 - sparse_categorical_accuracy: 0.0095 - val_loss: 7.3049 - val_sparse_categorical_accuracy: 0.0070
Epoch 6/25
4/4 [==============================] - 34s 8s/step - loss: 7.2347 - sparse_categorical_accuracy: 0.0095 - v

In [32]:
model_json = model.to_json()
with open('../models/lstm128.json', 'w') as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights('../models/lstm128.h5')
print("Saved model to disk")

Saved model to disk


In [35]:
sequences_test = sequences.apply(lambda x: x[-maxlen:])

In [38]:
sequences_test = sequences_test.apply(lambda x: [0 for i in range(maxlen - len(x))] + x)

In [40]:
sequences_test

user_id
446882394249262         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
11853380549289372       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
29584184913820301       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
34799051736481083       [602, 2438, 3752, 4237, 3580, 4437, 1698, 747,...
69093689723640662       [1501, 1086, 1840, 3149, 2518, 1263, 3577, 222...
                                              ...                        
18404354560751472507    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
18411538209698156830    [4081, 2612, 2511, 4090, 4127, 2465, 993, 2069...
18418817688105644452    [3191, 3905, 3299, 3990, 2107, 2994, 1180, 303...
18424331842900371747    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
18432872642329820182    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
Name: categ_id, Length: 2055, dtype: object

In [44]:
test_users_in_sequences = sorted(set(sequences_test.index) & set(correct_1.keys()))

In [45]:
X_test = np.array(sequences_test[test_users_in_sequences].tolist())

In [46]:
%%time
from itertools import chain
batch_size = 2048*8
n_batches = int(X_test.shape[0]/batch_size) + 1
preds = []

for batch_ind in tqdm.tqdm(range(n_batches)):
    batch = X_test[batch_ind*batch_size: (batch_ind + 1)*batch_size]
    curr_preds = model.predict(batch)
    curr_preds = np.argsort(-curr_preds)[:, :10]
    curr_preds = [[cat_to_element_uid[x] for x in row] for row in curr_preds]
    preds.append([' '.join(map(lambda x: str(x), row)) for row in curr_preds])
    
preds = list(chain(*preds))

100%|██████████| 1/1 [00:05<00:00,  5.31s/it]

CPU times: user 8.12 s, sys: 1.64 s, total: 9.76 s
Wall time: 5.32 s


In [47]:
preds

['18254 19677 19490 17239 19479 19645 7347 19861 19724 17364',
 '18254 19677 19490 17239 19479 19645 17364 19724 19207 19861',
 '18254 19677 19490 17239 19479 19645 7347 19861 16238 18534',
 '18254 19677 19490 17239 19479 7347 16238 18534 19645 19861',
 '18254 19677 19490 17239 19479 19645 7347 19861 16238 18534',
 '18254 19677 19490 17239 19479 19645 7347 16238 19861 18534',
 '18254 19677 19490 17239 19479 19645 7347 16238 18534 19861',
 '18254 19677 19490 17239 19479 7347 19645 16238 19861 18534',
 '18254 19677 19490 17239 19479 19645 7347 17364 19724 19861',
 '18254 19677 19490 17239 19479 19645 17364 19724 7347 19861',
 '18254 19677 19490 17239 19479 19645 7347 19861 19724 16238',
 '18254 19677 19490 19479 17239 19645 17364 19207 19724 19861',
 '18254 19677 19490 17239 19479 7347 16238 18534 19861 19645',
 '18254 19677 19490 17239 19479 19645 7347 19861 16238 19724',
 '18254 19677 19490 17239 19479 7347 19645 16238 19861 18534',
 '18254 19677 19490 17239 19479 16238 7347 18534 1986

In [ ]:
sample_submission.index = sample_submission.user_id
sample_submission.primary_video_id[test_users_in_sequences] = preds
sample_submission.to_csv('submission_lstm.csv', header=True, index=False)